### _執行前說明_

本作業使用 R 語言 (3.6.3) 進行資料前處理。

In [48]:
library(RODBC)
library(RWeka)
library(arules)
library(knitr)

db <- odbcDriverConnect("Driver={Microsoft Access Driver (*.mdb, *.accdb)};
                        DBQ=.\\foodmart2000.mdb")

### 題目 1

請利用Weka 中的Apriori 演算法，從Foodmart資料庫的交易資料中，探勘符合Minimum Support = 0.0001且Minimum Confidence = 0.9的Association Rules，並列出Confidence最高的前10條Rules。若無法跑出結果，請簡述其原因。

### 資料前處理

In [39]:
sql <- "
select
    trim(str(ft.customer_id)) & '-' & trim(str(ft.time_id)) & '-' & trim(str(ft.store_id)) as tid,
    pc.product_category as item
from sales_fact_1998 as ft, product as pd, product_class as pc
where pd.product_id = ft.product_id
and pd.product_class_id = pc.product_class_id;"

ft <- as.data.frame(sqlQuery(db, sql))
ft[, 1] <- as.character(ft[, 1])
ft[, 2] <- as.character(ft[, 2])

head(ft)

In [ ]:
tids <- unique(ft$tid)
print(paste('tid length:', length(tids)))

max_len <- 0
for (tid in tids){
    items <- unique(ft[ft$tid==tid, 2])
    item_len <- length(items)
    max_len <- ifelse(item_len > max_len, item_len, max_len)
}
print(paste('max_len:', max_len))

header <- paste('I', 1:max_len, sep = '')
result_frame <- rbind(NULL)

for (tid in tids){
    items <- unique(ft[ft$tid==tid, 2])
    items_len <- length(items)
    diff_len <- max_len - items_len
    row <- c(items, rep('', diff_len))
    result_frame <- rbind(result_frame, row)
}

result_frame <- rbind(header, result_frame)
head(result_frame)

write.arff(result_frame, file = 'sales_fact_1998_tx2.arff')
write.table(
    result_frame,
    'sales_fact_1998_tx2.csv',
    row.names = FALSE,
    col.names = FALSE,
    sep = ",",
    quote = TRUE)

### Weka 結果

根據上述程式做出之資料集進行 mining 後，得到以下十條 rule:

Idx | LHS | RHS | Conf | Lift
--- | --- | --- | ---- | ----
 1. | Magazines, Dairy, Snack Foods | Bread | 1 | 34.94
 2. | Magazines, Bread, Snack Foods | Dairy | 1 | 20.88
 3. | Snack Foods, Beer and Wine, Meat | Vegetables | 1 | 8.26
 4. | Jams and Jellies, Snack Foods, Hot Beverages | Dairy | 1 | 20.88
 5. | Magazines, Candy, Dairy | Snack Foods | 1 | 9.8
 6. | Starchy Foods, Breakfast Foods, Magazines | Snack Foods | 1 | 9.8
 7. | Starchy Foods, Snack Foods, Magazines | Breakfast Foods | 1 | 38.85
 8. | Snack Foods, Bread, Eggs | Vegetables | 1 | 8.26
 9. | Dairy, Snack Foods, Hygiene | Vegetables | 1 | 8.26
10. | Vegetables, Dairy, Hygiene | Snack Foods | 1 | 17.9

以下是 Weka 輸出結果：

```
=== Run information ===

Scheme:       weka.associations.Apriori -N 10 -T 0 -C 0.9 -D 1.0E-4 -U 1.0 -M 1.0E-4 -S -1.0 -V -c -1
Relation:     sales_fact_1998_tx2
Instances:    34070
Attributes:   18
              I1
              I2
              I3
              I4
              I5
              I6
              I7
              I8
              I9
              I10
              I11
              I12
              I13
              I14
              I15
              I16
              I17
              I18
=== Associator model (full training set) ===


Apriori
=======

Minimum support: 0 (7 instances)
Minimum metric <confidence>: 0.9
Number of cycles performed: 9998

Generated sets of large itemsets:

Size of set of large itemsets L(1): 452

Size of set of large itemsets L(2): 11408

Size of set of large itemsets L(3): 2766

Size of set of large itemsets L(4): 38

Best rules found:

 1. I1=Magazines I3=Dairy I5=Snack Foods 10 ==> I2=Bread 10    <conf:(1)> lift:(34.94) lev:(0) [9] conv:(9.71)
 2. I1=Magazines I2=Bread I5=Snack Foods 10 ==> I3=Dairy 10    <conf:(1)> lift:(20.88) lev:(0) [9] conv:(9.52)
 3. I2=Snack Foods I3=Beer and Wine I4=Meat 8 ==> I1=Vegetables 8    <conf:(1)> lift:(8.26) lev:(0) [7] conv:(7.03)
 4. I1=Jams and Jellies I2=Snack Foods I4=Hot Beverages 8 ==> I3=Dairy 8    <conf:(1)> lift:(20.88) lev:(0) [7] conv:(7.62)
 5. I1=Magazines I3=Candy I4=Dairy 8 ==> I2=Snack Foods 8    <conf:(1)> lift:(9.8) lev:(0) [7] conv:(7.18)
 6. I1=Starchy Foods I3=Breakfast Foods I4=Magazines 8 ==> I2=Snack Foods 8    <conf:(1)> lift:(9.8) lev:(0) [7] conv:(7.18)
 7. I1=Starchy Foods I2=Snack Foods I4=Magazines 8 ==> I3=Breakfast Foods 8    <conf:(1)> lift:(38.85) lev:(0) [7] conv:(7.79)
 8. I2=Snack Foods I3=Bread I4=Eggs 7 ==> I1=Vegetables 7    <conf:(1)> lift:(8.26) lev:(0) [6] conv:(6.15)
 9. I2=Dairy I4=Snack Foods I6=Hygiene 7 ==> I1=Vegetables 7    <conf:(1)> lift:(8.26) lev:(0) [6] conv:(6.15)
10. I1=Vegetables I2=Dairy I6=Hygiene 7 ==> I4=Snack Foods 7    <conf:(1)> lift:(17.9) lev:(0) [6] conv:(6.61)
```

### 題目 2

請利用Weka 中的FP-Growth演算法，從Foodmart資料庫的交易資料中，探勘符合Minimum Support = 0.0001 且Minimum Confidence = 0.9的Association Rules，並列出Confidence最高的前10條Rules。若無法跑出結果，請簡述其原因。

### 資料前處理

In [ ]:
sql2 <- "
select
    trim(str(ft.customer_id)) & '-' & trim(str(ft.time_id)) & '-' & trim(str(ft.store_id)) as tid,
    pc.product_category as item
from sales_fact_1998 as ft, product as pd, product_class as pc
where pd.product_id = ft.product_id
and pd.product_class_id = pc.product_class_id;"

ft <- as.data.frame(sqlQuery(db, sql2))
ft[, 1] <- as.character(ft[, 1])
ft[, 2] <- as.character(ft[, 2])

head(ft)

In [ ]:
sql_pc <- "select distinct product_category from product_class; "

pc <- as.data.frame(sqlQuery(db, sql_pc))
pc[,1] <- as.character(pc[,1])

columns <- c("tid", as.vector(t(pc)[1,]))
len <- length(columns)
tids <- unique(ft$tid)
df <- as.data.frame(matrix(ncol = length(columns), 
                           nrow=0, 
                           dimnames = list(NULL,columns)))

ridx <- 1

for(tid in tids){
    df[ridx, 1] = tid
    df[ridx, 2:len] = 0

    pcs <- unique(as.vector(ft[ft$tid==tid,2]))
    
    for(pc in pcs){
        df[ridx, pc] = 1
    }
    
    ridx <- ridx + 1
}

for (idx in 1:length(columns)){
    df[,idx] <- as.factor(df[,idx])
}

head(df)

write.arff(df[,2:len], file = 'sales_fact_1998_fp.arff')
write.table(
    df[,2:len],
    'sales_fact_1998_fp.csv',
    row.names = FALSE,
    col.names = TRUE,
    sep = ",",
    quote = FALSE)

### Weka 結果

根據上述程式做出之資料集進行 mining 後，得到以下十條 rule:

Idx | LHS | RHS | Conf | Lift
--- | --- | --- | ---- | ----
 1. | Meat, Fruit, Candy, Kitchen Products | Vegetables | 1 | 2.29
 2. | Meat, Breakfast Foods, Baking Goods, Specialty | Vegetables | 94% | 2.17
 3. | Jams and Jellies, Beer and Wine, Paper Products, Kitchen Products | Vegetables | 94% | 2.16
 4. | Snack Foods, Dairy, Jams and Jellies, Beer and Wine, Magazines | Vegetables | 94% | 2.16
 5. | Snack Foods, Fruit, Pain Relievers, Pizza | Vegetables | 94% | 2.15
 6. | Dairy, Meat, Baking Goods, Frozen Entrees | Vegetables | 94% | 2.15
 7. | Vegetables, Dairy, Canned Soup, Candy, Beer and Wine | Snack Foods | 94% | 2.23
 8. | Snack Foods, Meat, Jams and Jellies, Miscellaneous | Vegetables | 93% | 2.14
 9. | Meat, Breakfast Foods, Electrical, Pizza | Vegetables | 93% | 2.14
10. | Fruit, Candy, Electrical, Hot Beverages | Vegetables | 93% | 2.14

以下是 Weka 輸出結果：

```
=== Run information ===

Scheme:       weka.associations.FPGrowth -P 2 -I -1 -N 10 -T 0 -C 0.9 -D 1.0E-4 -U 1.0 -M 1.0E-4
Relation:     R_data_frame
Instances:    34070
Attributes:   47
              Baking Goods
              Bathroom Products
              Beer and Wine
              Bread
              Breakfast Foods
              Candles
              Candy
              Canned Anchovies
              Canned Clams
              Canned Oysters
              Canned Sardines
              Canned Shrimp
              Canned Soup
              Canned Tuna
              Carbonated Beverages
              Cleaning Supplies
              Cold Remedies
              Dairy
              Decongestants
              Drinks
              Dry Goods
              Eggs
              Electrical
              Frozen Desserts
              Frozen Entrees
              Fruit
              Hardware
              Hot Beverages
              Hygiene
              Jams and Jellies
              Kitchen Products
              Magazines
              Meat
              Miscellaneous
              Packaged Soup
              Packaged Vegetables
              Pain Relievers
              Paper Products
              Pizza
              Plastic Products
              Pure Juice Beverages
              Seafood
              Side Dishes
              Snack Foods
              Specialty
              Starchy Foods
              Vegetables
=== Associator model (full training set) ===

FPGrowth found 17 rules (displaying top 10)

 1. [Meat=1, Fruit=1, Candy=1, Kitchen Products=1]: 14 ==> [Vegetables=1]: 14   <conf:(1)> lift:(2.29) lev:(0) conv:(7.89) 
 2. [Meat=1, Breakfast Foods=1, Baking Goods=1, Specialty=1]: 18 ==> [Vegetables=1]: 17   <conf:(0.94)> lift:(2.17) lev:(0) conv:(5.07) 
 3. [Jams and Jellies=1, Beer and Wine=1, Paper Products=1, Kitchen Products=1]: 17 ==> [Vegetables=1]: 16   <conf:(0.94)> lift:(2.16) lev:(0) conv:(4.79) 
 4. [Snack Foods=1, Dairy=1, Jams and Jellies=1, Beer and Wine=1, Magazines=1]: 17 ==> [Vegetables=1]: 16   <conf:(0.94)> lift:(2.16) lev:(0) conv:(4.79) 
 5. [Snack Foods=1, Fruit=1, Pain Relievers=1, Pizza=1]: 16 ==> [Vegetables=1]: 15   <conf:(0.94)> lift:(2.15) lev:(0) conv:(4.51) 
 6. [Dairy=1, Meat=1, Baking Goods=1, Frozen Entrees=1]: 16 ==> [Vegetables=1]: 15   <conf:(0.94)> lift:(2.15) lev:(0) conv:(4.51) 
 7. [Vegetables=1, Dairy=1, Canned Soup=1, Candy=1, Beer and Wine=1]: 16 ==> [Snack Foods=1]: 15   <conf:(0.94)> lift:(2.23) lev:(0) conv:(4.64) 
 8. [Snack Foods=1, Meat=1, Jams and Jellies=1, Miscellaneous=1]: 15 ==> [Vegetables=1]: 14   <conf:(0.93)> lift:(2.14) lev:(0) conv:(4.23) 
 9. [Meat=1, Breakfast Foods=1, Electrical=1, Pizza=1]: 15 ==> [Vegetables=1]: 14   <conf:(0.93)> lift:(2.14) lev:(0) conv:(4.23) 
10. [Fruit=1, Candy=1, Electrical=1, Hot Beverages=1]: 15 ==> [Vegetables=1]: 14   <conf:(0.93)> lift:(2.14) lev:(0) conv:(4.23) 
```

### 題目 3

有時候我們有興趣的資料不只有產品間的資訊，也會想要由User Profile探勘顧客的基本資料。請運用Weka，給定Minimum Support = 0.05且Minimum Confidence= 0.9的條件下，探勘Foodmart顧客基本資料的屬性{State_Province, Yearly_Income , Gender , Total_Children , Num_Children_at_Home , Education , Occupation, Houseowner , Num_cars,owned } 間的association rule。(列出10條)

### 資料前處理

In [ ]:
db <- odbcDriverConnect("Driver={Microsoft Access Driver (*.mdb, *.accdb)};
                        DBQ=.\\foodmart2000.mdb")

sql3 <- "
select distinct
    ct.customer_id,
    ct.state_province,
    ct.yearly_income,
    ct.gender,
    ct.total_children,
    ct.num_children_at_home,
    ct.education,
    ct.occupation,
    ct.houseowner,
    ct.num_cars_owned
from sales_fact_1998 tx, customer ct
where tx.customer_id = ct.customer_id;
"

ct <- as.data.frame(sqlQuery(db, sql3))
for(idx in 2:10){
    ct[,idx] <- as.factor(ct[, idx])
}

head(ct)

write.arff(ct[,2:10], file = 'customer.arff')
write.table(
    df[,2:10],
    'customer.csv',
    row.names = FALSE,
    col.names = TRUE,
    sep = ",",
    quote = FALSE)

### Weka 結果

根據上述程式做出之資料集進行 mining 後，得到了許多結果，但排序在前段的有一些是沒有必要的，故過濾之後的前七項如下：

 Idx | LHS | RHS | Conf | Lift
 --- | --- | --- | ---- | ----
 3. | num_cars_owned=0 | yearly_income=$10K - $30K | 1 | 4.66%
29. | yearly_income=$10K - $30K | education=Partial High School | 0.92 | 3.13%
11. | yearly_income=$10K - $30K occupation=Manual | education=Partial High School | 0.96 | 3.26
13. | yearly_income=$10K - $30K occupation=Skilled Manual | education=Partial High School | 0.95 | 3.25
22. | yearly_income=$10K - $30K houseowner=Y | education=Partial High School | 0.93 | 3.16
15. | yearly_income=$50K - $70K occupation=Professional | education=Bachelors Degree | 0.95 | 3.69
41. | education=High School Degree occupation=Manual | yearly_income=$30K - $50K | 0.9 | 2.75

無法做出前十項結果的原因在於，mining 結果有許多是集合的重疊，因此我選擇了較具識別性，也比較寬鬆的條件做為結果。

Weka 的完整輸出如下：

```
=== Run information ===

Scheme:       weka.associations.Apriori -N 50 -T 0 -C 0.9 -D 0.05 -U 1.0 -M 0.05 -S -1.0 -c -1
Relation:     R_data_frame
Instances:    7824
Attributes:   9
              state_province
              yearly_income
              gender
              total_children
              num_children_at_home
              education
              occupation
              houseowner
              num_cars_owned
=== Associator model (full training set) ===


Apriori
=======

Minimum support: 0.05 (391 instances)
Minimum metric <confidence>: 0.9
Number of cycles performed: 19

Generated sets of large itemsets:

Size of set of large itemsets L(1): 36

Size of set of large itemsets L(2): 243

Size of set of large itemsets L(3): 274

Size of set of large itemsets L(4): 47

Best rules found:

 1. total_children=0 791 ==> num_children_at_home=0 791    <conf:(1)> lift:(1.58) lev:(0.04) [290] conv:(290.76)
 2. total_children=0 houseowner=Y 486 ==> num_children_at_home=0 486    <conf:(1)> lift:(1.58) lev:(0.02) [178] conv:(178.65)
 3. num_cars_owned=0 445 ==> yearly_income=$10K - $30K 445    <conf:(1)> lift:(4.66) lev:(0.04) [349] conv:(349.5)
 4. gender=M total_children=0 425 ==> num_children_at_home=0 425    <conf:(1)> lift:(1.58) lev:(0.02) [156] conv:(156.22)
 5. education=Partial High School num_cars_owned=0 408 ==> yearly_income=$10K - $30K 408    <conf:(1)> lift:(4.66) lev:(0.04) [320] conv:(320.44)
 6. yearly_income=$10K - $30K gender=M occupation=Skilled Manual 405 ==> education=Partial High School 393    <conf:(0.97)> lift:(3.3) lev:(0.04) [273] conv:(22)
 7. yearly_income=$10K - $30K occupation=Manual houseowner=Y 480 ==> education=Partial High School 463    <conf:(0.96)> lift:(3.28) lev:(0.04) [321] conv:(18.83)
 8. yearly_income=$10K - $30K occupation=Skilled Manual houseowner=Y 463 ==> education=Partial High School 446    <conf:(0.96)> lift:(3.28) lev:(0.04) [309] conv:(18.16)
 9. yearly_income=$10K - $30K num_children_at_home=0 occupation=Manual 510 ==> education=Partial High School 491    <conf:(0.96)> lift:(3.28) lev:(0.04) [341] conv:(18)
10. yearly_income=$10K - $30K gender=M occupation=Manual 419 ==> education=Partial High School 402    <conf:(0.96)> lift:(3.26) lev:(0.04) [278] conv:(16.43)
11. yearly_income=$10K - $30K occupation=Manual 807 ==> education=Partial High School 774    <conf:(0.96)> lift:(3.26) lev:(0.07) [536] conv:(16.76)
12. yearly_income=$10K - $30K occupation=Skilled Manual num_cars_owned=1 420 ==> education=Partial High School 401    <conf:(0.95)> lift:(3.25) lev:(0.04) [277] conv:(14.83)
13. yearly_income=$10K - $30K occupation=Skilled Manual 810 ==> education=Partial High School 773    <conf:(0.95)> lift:(3.25) lev:(0.07) [534] conv:(15.05)
14. yearly_income=$10K - $30K num_children_at_home=0 occupation=Skilled Manual 500 ==> education=Partial High School 477    <conf:(0.95)> lift:(3.25) lev:(0.04) [330] conv:(14.71)
15. yearly_income=$50K - $70K occupation=Professional 798 ==> education=Bachelors Degree 758    <conf:(0.95)> lift:(3.69) lev:(0.07) [552] conv:(14.46)
16. yearly_income=$50K - $70K occupation=Professional houseowner=Y 455 ==> education=Bachelors Degree 429    <conf:(0.94)> lift:(3.67) lev:(0.04) [311] conv:(12.52)
17. yearly_income=$50K - $70K num_children_at_home=0 occupation=Professional 518 ==> education=Bachelors Degree 487    <conf:(0.94)> lift:(3.66) lev:(0.05) [353] conv:(12.02)
18. education=Partial High School occupation=Skilled Manual num_cars_owned=1 427 ==> yearly_income=$10K - $30K 401    <conf:(0.94)> lift:(4.38) lev:(0.04) [309] conv:(12.42)
19. yearly_income=$10K - $30K num_children_at_home=0 houseowner=Y 501 ==> education=Partial High School 470    <conf:(0.94)> lift:(3.19) lev:(0.04) [322] conv:(11.05)
20. yearly_income=$10K - $30K houseowner=Y num_cars_owned=1 484 ==> education=Partial High School 453    <conf:(0.94)> lift:(3.18) lev:(0.04) [310] conv:(10.68)
21. yearly_income=$10K - $30K gender=M houseowner=Y 508 ==> education=Partial High School 475    <conf:(0.94)> lift:(3.18) lev:(0.04) [325] conv:(10.55)
22. yearly_income=$10K - $30K houseowner=Y 977 ==> education=Partial High School 909    <conf:(0.93)> lift:(3.16) lev:(0.08) [621] conv:(10)
23. yearly_income=$10K - $30K gender=M 857 ==> education=Partial High School 795    <conf:(0.93)> lift:(3.16) lev:(0.07) [543] conv:(9.6)
24. yearly_income=$10K - $30K gender=F houseowner=Y 469 ==> education=Partial High School 434    <conf:(0.93)> lift:(3.15) lev:(0.04) [296] conv:(9.2)
25. yearly_income=$10K - $30K gender=F num_children_at_home=0 503 ==> education=Partial High School 465    <conf:(0.92)> lift:(3.14) lev:(0.04) [317] conv:(9.11)
26. yearly_income=$10K - $30K num_children_at_home=0 1048 ==> education=Partial High School 968    <conf:(0.92)> lift:(3.14) lev:(0.08) [659] conv:(9.13)
27. yearly_income=$10K - $30K num_cars_owned=1 837 ==> education=Partial High School 773    <conf:(0.92)> lift:(3.14) lev:(0.07) [526] conv:(9.09)
28. yearly_income=$10K - $30K gender=M num_children_at_home=0 545 ==> education=Partial High School 503    <conf:(0.92)> lift:(3.14) lev:(0.04) [342] conv:(8.95)
29. yearly_income=$10K - $30K 1679 ==> education=Partial High School 1547    <conf:(0.92)> lift:(3.13) lev:(0.13) [1053] conv:(8.91)
30. yearly_income=$10K - $30K num_children_at_home=0 num_cars_owned=1 514 ==> education=Partial High School 472    <conf:(0.92)> lift:(3.12) lev:(0.04) [320] conv:(8.44)
31. num_cars_owned=0 445 ==> education=Partial High School 408    <conf:(0.92)> lift:(3.12) lev:(0.04) [277] conv:(8.27)
32. yearly_income=$10K - $30K num_cars_owned=0 445 ==> education=Partial High School 408    <conf:(0.92)> lift:(3.12) lev:(0.04) [277] conv:(8.27)
33. num_cars_owned=0 445 ==> yearly_income=$10K - $30K education=Partial High School 408    <conf:(0.92)> lift:(4.64) lev:(0.04) [320] conv:(9.4)
34. yearly_income=$10K - $30K gender=F 822 ==> education=Partial High School 752    <conf:(0.91)> lift:(3.11) lev:(0.07) [510] conv:(8.17)
35. state_province=CA yearly_income=$10K - $30K 547 ==> education=Partial High School 499    <conf:(0.91)> lift:(3.1) lev:(0.04) [338] conv:(7.88)
36. gender=M education=High School Degree occupation=Manual 470 ==> yearly_income=$30K - $50K 428    <conf:(0.91)> lift:(2.78) lev:(0.04) [274] conv:(7.35)
37. yearly_income=$10K - $30K num_children_at_home=0 houseowner=N 547 ==> education=Partial High School 498    <conf:(0.91)> lift:(3.1) lev:(0.04) [337] conv:(7.72)
38. yearly_income=$10K - $30K houseowner=N 702 ==> education=Partial High School 638    <conf:(0.91)> lift:(3.09) lev:(0.06) [431] conv:(7.63)
39. num_children_at_home=0 education=High School Degree occupation=Manual 583 ==> yearly_income=$30K - $50K 529    <conf:(0.91)> lift:(2.77) lev:(0.04) [338] conv:(7.13)
40. gender=M education=High School Degree occupation=Skilled Manual 457 ==> yearly_income=$30K - $50K 413    <conf:(0.9)> lift:(2.76) lev:(0.03) [263] conv:(6.83)
41. education=High School Degree occupation=Manual 908 ==> yearly_income=$30K - $50K 819    <conf:(0.9)> lift:(2.75) lev:(0.07) [521] conv:(6.79)
```

### 題目 4

請運用Weka探勘Foodmart資料庫中，顧客背景資料與其交易資料之間的關係(Quantitative Association Rules)。例如80%女性顧客常買保養品。請自行嘗試設定Minimum Support Minimum Confidence，找出10條你覺得有意義的Rules。請說明你的作法及相關參數設定。

### 資料前處理

In [ ]:
db <- odbcDriverConnect("Driver={Microsoft Access Driver (*.mdb, *.accdb)};
                        DBQ=.\\foodmart2000.mdb")

sql4 <- "
select
    trim(str(ft.customer_id)) & '-' & trim(str(ft.time_id)) & '-' & trim(str(ft.store_id)) as tid,
    ct.yearly_income,
    ct.gender,
    ct.occupation,
    pc.product_category as item
from sales_fact_1998 as ft, product as pd, product_class as pc, customer as ct
where pd.product_id = ft.product_id
and pd.product_class_id = pc.product_class_id
and ft.customer_id = ct.customer_id;"

max_columns <- 5

ft <- as.data.frame(sqlQuery(db, sql4))

for(ci in 1:max_columns){
   ft[,ci] <- as.character(ft[,ci])
}

head(ft)

In [ ]:
columns <- c("tid")

cate_sql <- "select distinct product_category from product_class;"

cust_attrs <- c("yearly_income", "gender", "education", "occupation")

attr_sqls = paste("select distinct", cust_attrs, "from customer;")

all_attr_sqls = c(attr_sqls, cate_sql)

for( sql in all_attr_sqls){
    pc <- as.data.frame(sqlQuery(db, sql))
    pc[,1] <- as.character(pc[,1])
    columns <- c(columns, as.vector(t(pc)[1,]))
}

len <- length(columns)
# tids <- head(unique(ft$tid), n=5000)
tids <- unique(ft$tid)
df <- as.data.frame(matrix(ncol = length(columns), 
                           nrow=0, 
                           dimnames = list(NULL,columns)))

ridx <- 1
for(tid in tids){
    df[ridx, 1] = tid
    df[ridx, 2:len] = 0

    cust_indices <- 2:(length(cust_attrs) + 1)
    for(cust_idx in cust_indices){
        ca <- unique(as.vector(ft[ft$tid==tid, cust_idx]))
        df[ridx, ca] = 1
    }
    
    prod_idx <- max_columns
    pcs <- unique(as.vector(ft[ft$tid==tid, prod_idx]))    
    for(pc in pcs){
        df[ridx, pc] = 1
    }
    
    ridx <- ridx + 1
}

for (idx in 1:length(columns)){
    df[,idx] <- as.factor(df[,idx])
}

head(df)

write.arff(df[,2:len], file = 'cust_tx_fp.arff')
write.table(
    df[,2:len],
    'cust_tx_fp.csv',
    row.names = FALSE,
    col.names = TRUE,
    sep = ",",
    quote = FALSE)

### Weka 結果

這次的 mining，選擇了從客戶特性分析中的幾項具代表性的 feature：性別、年收、教育、職業，使用 FP-Growth (效率高)，並基於 minimum support 0.0005 的情況，選出前 100 個 rule，再加以過濾出以下 10 個 rule：

 Idx | LHS | RHS | Conf | Lift
 --- | --- | --- | ---- | ----
 

Weka 輸出結果：

```
=== Run information ===

Scheme:       weka.associations.FPGrowth -P 2 -I -1 -N 100 -T 0 -C 0.9 -D 5.0E-4 -U 1.0 -M 5.0E-4
Relation:     R_data_frame
Instances:    34070
Attributes:   67
              $10K - $30K
              $110K - $130K
              $130K - $150K
              $150K +
              $30K - $50K
              $50K - $70K
              $70K - $90K
              $90K - $110K
              F
              M
              Bachelors Degree
              Graduate Degree
              High School Degree
              Partial College
              Partial High School
              Clerical
              Management
              Manual
              Professional
              Skilled Manual
              Baking Goods
              Bathroom Products
              Beer and Wine
              Bread
              Breakfast Foods
              Candles
              Candy
              Canned Anchovies
              Canned Clams
              Canned Oysters
              Canned Sardines
              Canned Shrimp
              Canned Soup
              Canned Tuna
              Carbonated Beverages
              Cleaning Supplies
              Cold Remedies
              Dairy
              Decongestants
              Drinks
              Dry Goods
              Eggs
              Electrical
              Frozen Desserts
              Frozen Entrees
              Fruit
              Hardware
              Hot Beverages
              Hygiene
              Jams and Jellies
              Kitchen Products
              Magazines
              Meat
              Miscellaneous
              Packaged Soup
              Packaged Vegetables
              Pain Relievers
              Paper Products
              Pizza
              Plastic Products
              Pure Juice Beverages
              Seafood
              Side Dishes
              Snack Foods
              Specialty
              Starchy Foods
              Vegetables
=== Associator model (full training set) ===

FPGrowth found 96 rules (displaying top 96)

 1. [F=1, Pure Juice Beverages=1, Clerical=1]: 22 ==> [$30K - $50K=1]: 22   <conf:(1)> lift:(3.06) lev:(0) conv:(14.82) 
 2. [Meat=1, Jams and Jellies=1, Clerical=1]: 20 ==> [$30K - $50K=1]: 20   <conf:(1)> lift:(3.06) lev:(0) conv:(13.47) 
 3. [Meat=1, Beer and Wine=1, Clerical=1]: 19 ==> [$30K - $50K=1]: 19   <conf:(1)> lift:(3.06) lev:(0) conv:(12.8) 
 4. [Dairy=1, $130K - $150K=1, Hardware=1]: 21 ==> [Professional=1]: 21   <conf:(1)> lift:(3.08) lev:(0) conv:(14.19) 
 5. [F=1, Snack Foods=1, Electrical=1, Clerical=1]: 19 ==> [$30K - $50K=1]: 19   <conf:(1)> lift:(3.06) lev:(0) conv:(12.8) 
 6. [Vegetables=1, Snack Foods=1, Meat=1, Clerical=1]: 28 ==> [$30K - $50K=1]: 28   <conf:(1)> lift:(3.06) lev:(0) conv:(18.86) 
 7. [F=1, Bathroom Products=1, Clerical=1]: 38 ==> [$30K - $50K=1]: 37   <conf:(0.97)> lift:(2.98) lev:(0) conv:(12.8) 
 8. [F=1, Electrical=1, Clerical=1]: 33 ==> [$30K - $50K=1]: 32   <conf:(0.97)> lift:(2.97) lev:(0) conv:(11.11) 
 9. [F=1, Vegetables=1, Meat=1, Clerical=1]: 33 ==> [$30K - $50K=1]: 32   <conf:(0.97)> lift:(2.97) lev:(0) conv:(11.11) 
10. [F=1, Snack Foods=1, Meat=1, Clerical=1]: 26 ==> [$30K - $50K=1]: 25   <conf:(0.96)> lift:(2.94) lev:(0) conv:(8.76) 
11. [F=1, Frozen Desserts=1, Clerical=1]: 24 ==> [$30K - $50K=1]: 23   <conf:(0.96)> lift:(2.94) lev:(0) conv:(8.08) 
12. [Vegetables=1, Jams and Jellies=1, Clerical=1]: 48 ==> [$30K - $50K=1]: 46   <conf:(0.96)> lift:(2.94) lev:(0) conv:(10.78) 
13. [F=1, Candy=1, Clerical=1]: 22 ==> [$30K - $50K=1]: 21   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.41) 
14. [Vegetables=1, Meat=1, Clerical=1]: 66 ==> [$30K - $50K=1]: 63   <conf:(0.95)> lift:(2.92) lev:(0) conv:(11.11) 
15. [Dairy=1, Bread=1, Clerical=1]: 22 ==> [$30K - $50K=1]: 21   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.41) 
16. [Dairy=1, Baking Goods=1, $150K +=1]: 22 ==> [Professional=1]: 21   <conf:(0.95)> lift:(2.94) lev:(0) conv:(7.43) 
17. [F=1, Snack Foods=1, Bathroom Products=1, Clerical=1]: 22 ==> [$30K - $50K=1]: 21   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.41) 
18. [Snack Foods=1, Hardware=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 20   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.07) 
19. [Meat=1, Bathroom Products=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 20   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.07) 
20. [F=1, Dairy=1, Meat=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 20   <conf:(0.95)> lift:(2.92) lev:(0) conv:(7.07) 
21. [$110K - $130K=1, Canned Clams=1]: 20 ==> [Professional=1]: 19   <conf:(0.95)> lift:(2.93) lev:(0) conv:(6.76) 
22. [Snack Foods=1, Carbonated Beverages=1, $150K +=1]: 20 ==> [Professional=1]: 19   <conf:(0.95)> lift:(2.93) lev:(0) conv:(6.76) 
23. [Dairy=1, Bathroom Products=1, Clerical=1]: 20 ==> [$30K - $50K=1]: 19   <conf:(0.95)> lift:(2.91) lev:(0) conv:(6.73) 
24. [Bread=1, Paper Products=1, $110K - $130K=1]: 20 ==> [Professional=1]: 19   <conf:(0.95)> lift:(2.93) lev:(0) conv:(6.76) 
25. [Canned Soup=1, Eggs=1, $130K - $150K=1]: 19 ==> [Professional=1]: 18   <conf:(0.95)> lift:(2.92) lev:(0) conv:(6.42) 
26. [F=1, Snack Foods=1, Jams and Jellies=1, Clerical=1]: 19 ==> [$30K - $50K=1]: 18   <conf:(0.95)> lift:(2.9) lev:(0) conv:(6.4) 
27. [M=1, Vegetables=1, Dairy=1, Candy=1, Eggs=1]: 19 ==> [Snack Foods=1]: 18   <conf:(0.95)> lift:(2.25) lev:(0) conv:(5.5) 
28. [Snack Foods=1, Skilled Manual=1, Dairy=1, Candy=1, Electrical=1]: 19 ==> [$30K - $50K=1]: 18   <conf:(0.95)> lift:(2.9) lev:(0) conv:(6.4) 
29. [F=1, Jams and Jellies=1, Clerical=1]: 37 ==> [$30K - $50K=1]: 35   <conf:(0.95)> lift:(2.9) lev:(0) conv:(8.31) 
30. [F=1, Baking Goods=1, Clerical=1]: 51 ==> [$30K - $50K=1]: 48   <conf:(0.94)> lift:(2.88) lev:(0) conv:(8.59) 
31. [F=1, Bread=1, Clerical=1]: 33 ==> [$30K - $50K=1]: 31   <conf:(0.94)> lift:(2.88) lev:(0) conv:(7.41) 
32. [M=1, Vegetables=1, Meat=1, Clerical=1]: 33 ==> [$30K - $50K=1]: 31   <conf:(0.94)> lift:(2.88) lev:(0) conv:(7.41) 
33. [F=1, Meat=1, Clerical=1]: 64 ==> [$30K - $50K=1]: 60   <conf:(0.94)> lift:(2.87) lev:(0) conv:(8.62) 
34. [M=1, Vegetables=1, Jams and Jellies=1, Clerical=1]: 32 ==> [$30K - $50K=1]: 30   <conf:(0.94)> lift:(2.87) lev:(0) conv:(7.18) 
35. [Meat=1, Fruit=1, Clerical=1]: 30 ==> [$30K - $50K=1]: 28   <conf:(0.93)> lift:(2.86) lev:(0) conv:(6.73) 
36. [Canned Soup=1, Electrical=1, $130K - $150K=1]: 30 ==> [Professional=1]: 28   <conf:(0.93)> lift:(2.88) lev:(0) conv:(6.76) 
37. [Snack Foods=1, Bathroom Products=1, Clerical=1]: 42 ==> [$30K - $50K=1]: 39   <conf:(0.93)> lift:(2.84) lev:(0) conv:(7.07) 
38. [Snack Foods=1, Meat=1, Clerical=1]: 55 ==> [$30K - $50K=1]: 51   <conf:(0.93)> lift:(2.84) lev:(0) conv:(7.41) 
39. [Snack Foods=1, Paper Products=1, Clerical=1]: 27 ==> [$30K - $50K=1]: 25   <conf:(0.93)> lift:(2.84) lev:(0) conv:(6.06) 
40. [Vegetables=1, Snack Foods=1, Jams and Jellies=1, Clerical=1]: 27 ==> [$30K - $50K=1]: 25   <conf:(0.93)> lift:(2.84) lev:(0) conv:(6.06) 
41. [Vegetables=1, Dairy=1, Meat=1, Clerical=1]: 27 ==> [$30K - $50K=1]: 25   <conf:(0.93)> lift:(2.84) lev:(0) conv:(6.06) 
42. [Dairy=1, Meat=1, Clerical=1]: 39 ==> [$30K - $50K=1]: 36   <conf:(0.92)> lift:(2.83) lev:(0) conv:(6.57) 
43. [F=1, Fruit=1, Clerical=1]: 63 ==> [$30K - $50K=1]: 58   <conf:(0.92)> lift:(2.82) lev:(0) conv:(7.07) 
44. [Drinks=1, Clerical=1]: 25 ==> [$30K - $50K=1]: 23   <conf:(0.92)> lift:(2.82) lev:(0) conv:(5.61) 
45. [F=1, Vegetables=1, Breakfast Foods=1, $150K +=1]: 25 ==> [Professional=1]: 23   <conf:(0.92)> lift:(2.84) lev:(0) conv:(5.63) 
46. [M=1, Meat=1, Bread=1, $110K - $130K=1]: 25 ==> [Professional=1]: 23   <conf:(0.92)> lift:(2.84) lev:(0) conv:(5.63) 
47. [F=1, Snack Foods=1, Meat=1, $70K - $90K=1, Frozen Desserts=1]: 25 ==> [Professional=1]: 23   <conf:(0.92)> lift:(2.84) lev:(0) conv:(5.63) 
48. [Jams and Jellies=1, Hygiene=1, Hardware=1]: 23 ==> [Snack Foods=1]: 21   <conf:(0.91)> lift:(2.17) lev:(0) conv:(4.44) 
49. [F=1, Jams and Jellies=1, Bathroom Products=1, Kitchen Products=1]: 23 ==> [Vegetables=1]: 21   <conf:(0.91)> lift:(2.09) lev:(0) conv:(4.32) 
50. [Snack Foods=1, Fruit=1, Paper Products=1, $90K - $110K=1]: 23 ==> [Professional=1]: 21   <conf:(0.91)> lift:(2.81) lev:(0) conv:(5.18) 
51. [Skilled Manual=1, Carbonated Beverages=1, Cold Remedies=1]: 22 ==> [$30K - $50K=1]: 20   <conf:(0.91)> lift:(2.78) lev:(0) conv:(4.94) 
52. [Fruit=1, Bread=1, Clerical=1]: 22 ==> [$30K - $50K=1]: 20   <conf:(0.91)> lift:(2.78) lev:(0) conv:(4.94) 
53. [F=1, Skilled Manual=1, Pizza=1, Cleaning Supplies=1]: 22 ==> [Vegetables=1]: 20   <conf:(0.91)> lift:(2.08) lev:(0) conv:(4.14) 
54. [F=1, Fruit=1, Breakfast Foods=1, $90K - $110K=1]: 22 ==> [Professional=1]: 20   <conf:(0.91)> lift:(2.8) lev:(0) conv:(4.95) 
55. [M=1, Snack Foods=1, Manual=1, Paper Products=1, Pain Relievers=1]: 22 ==> [Vegetables=1]: 20   <conf:(0.91)> lift:(2.08) lev:(0) conv:(4.14) 
56. [M=1, Vegetables=1, Meat=1, $70K - $90K=1, Frozen Desserts=1]: 22 ==> [Professional=1]: 20   <conf:(0.91)> lift:(2.8) lev:(0) conv:(4.95) 
57. [F=1, Snack Foods=1, Professional=1, Dairy=1, Fruit=1, Beer and Wine=1]: 22 ==> [Vegetables=1]: 20   <conf:(0.91)> lift:(2.08) lev:(0) conv:(4.14) 
58. [M=1, Baking Goods=1, $150K +=1]: 43 ==> [Professional=1]: 39   <conf:(0.91)> lift:(2.8) lev:(0) conv:(5.81) 
59. [F=1, Paper Products=1, Clerical=1]: 32 ==> [$30K - $50K=1]: 29   <conf:(0.91)> lift:(2.78) lev:(0) conv:(5.39) 
60. [Vegetables=1, Bread=1, Clerical=1]: 32 ==> [$30K - $50K=1]: 29   <conf:(0.91)> lift:(2.78) lev:(0) conv:(5.39) 
61. [Vegetables=1, Electrical=1, Clerical=1]: 32 ==> [$30K - $50K=1]: 29   <conf:(0.91)> lift:(2.78) lev:(0) conv:(5.39) 
62. [Meat=1, Clerical=1]: 126 ==> [$30K - $50K=1]: 114   <conf:(0.9)> lift:(2.77) lev:(0) conv:(6.53) 
63. [F=1, Carbonated Beverages=1, $150K +=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
64. [M=1, Hardware=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 19   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.71) 
65. [Vegetables=1, Eggs=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 19   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.71) 
66. [Fruit=1, Hygiene=1, Hardware=1]: 21 ==> [Snack Foods=1]: 19   <conf:(0.9)> lift:(2.15) lev:(0) conv:(4.06) 
67. [Fruit=1, Baking Goods=1, $150K +=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
68. [Bathroom Products=1, Paper Products=1, $90K - $110K=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
69. [M=1, Vegetables=1, Bread=1, Clerical=1]: 21 ==> [$30K - $50K=1]: 19   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.71) 
70. [M=1, Snack Foods=1, Carbonated Beverages=1, $110K - $130K=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
71. [Snack Foods=1, Bread=1, Bathroom Products=1, Drinks=1]: 21 ==> [Vegetables=1]: 19   <conf:(0.9)> lift:(2.07) lev:(0) conv:(3.95) 
72. [Vegetables=1, Breakfast Foods=1, Baking Goods=1, $130K - $150K=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
73. [F=1, Snack Foods=1, Dairy=1, $70K - $90K=1, Eggs=1]: 21 ==> [Professional=1]: 19   <conf:(0.9)> lift:(2.79) lev:(0) conv:(4.73) 
74. [M=1, Snack Foods=1, Fruit=1, Paper Products=1, Pain Relievers=1]: 21 ==> [Vegetables=1]: 19   <conf:(0.9)> lift:(2.07) lev:(0) conv:(3.95) 
75. [M=1, Snack Foods=1, $30K - $50K=1, Dairy=1, Manual=1, Pain Relievers=1]: 21 ==> [Vegetables=1]: 19   <conf:(0.9)> lift:(2.07) lev:(0) conv:(3.95) 
76. [F=1, Vegetables=1, Clerical=1]: 114 ==> [$30K - $50K=1]: 103   <conf:(0.9)> lift:(2.77) lev:(0) conv:(6.4) 
77. [Snack Foods=1, Bread=1, Clerical=1]: 31 ==> [$30K - $50K=1]: 28   <conf:(0.9)> lift:(2.77) lev:(0) conv:(5.22) 
78. [Vegetables=1, Snack Foods=1, Professional=1, Meat=1, $50K - $70K=1, Canned Soup=1]: 31 ==> [F=1]: 28   <conf:(0.9)> lift:(1.79) lev:(0) conv:(3.83) 
79. [Bread=1, Clerical=1]: 72 ==> [$30K - $50K=1]: 65   <conf:(0.9)> lift:(2.76) lev:(0) conv:(6.06) 
80. [Pure Juice Beverages=1, Clerical=1]: 40 ==> [$30K - $50K=1]: 36   <conf:(0.9)> lift:(2.76) lev:(0) conv:(5.39) 
81. [Bread=1, Hygiene=1, Frozen Entrees=1]: 20 ==> [F=1]: 18   <conf:(0.9)> lift:(1.78) lev:(0) conv:(3.3) 
82. [Bathroom Products=1, Hot Beverages=1, Hardware=1]: 20 ==> [Snack Foods=1]: 18   <conf:(0.9)> lift:(2.14) lev:(0) conv:(3.86) 
83. [Fruit=1, Frozen Desserts=1, $150K +=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
84. [Electrical=1, Paper Products=1, $110K - $130K=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
85. [F=1, Fruit=1, Kitchen Products=1, Pain Relievers=1]: 20 ==> [Vegetables=1]: 18   <conf:(0.9)> lift:(2.06) lev:(0) conv:(3.76) 
86. [F=1, Fruit=1, Bathroom Products=1, $90K - $110K=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
87. [M=1, Snack Foods=1, Bathroom Products=1, Clerical=1]: 20 ==> [$30K - $50K=1]: 18   <conf:(0.9)> lift:(2.76) lev:(0) conv:(4.49) 
88. [M=1, Jams and Jellies=1, Breakfast Foods=1, $110K - $130K=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
89. [M=1, Breakfast Foods=1, Starchy Foods=1, $130K - $150K=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
90. [Snack Foods=1, Jams and Jellies=1, Specialty=1, Plastic Products=1]: 20 ==> [Vegetables=1]: 18   <conf:(0.9)> lift:(2.06) lev:(0) conv:(3.76) 
91. [Snack Foods=1, Dairy=1, Jams and Jellies=1, Clerical=1]: 20 ==> [$30K - $50K=1]: 18   <conf:(0.9)> lift:(2.76) lev:(0) conv:(4.49) 
92. [Dairy=1, Bread=1, $70K - $90K=1, Magazines=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
93. [$10K - $30K=1, Breakfast Foods=1, Baking Goods=1, Pizza=1]: 20 ==> [Skilled Manual=1]: 18   <conf:(0.9)> lift:(3.33) lev:(0) conv:(4.87) 
94. [M=1, Vegetables=1, Fruit=1, Breakfast Foods=1, $130K - $150K=1]: 20 ==> [Professional=1]: 18   <conf:(0.9)> lift:(2.77) lev:(0) conv:(4.5) 
95. [Snack Foods=1, Dairy=1, Meat=1, Fruit=1, Pizza=1]: 20 ==> [Vegetables=1]: 18   <conf:(0.9)> lift:(2.06) lev:(0) conv:(3.76) 
96. [Vegetables=1, Dairy=1, Jams and Jellies=1, Candy=1, Beer and Wine=1]: 20 ==> [Snack Foods=1]: 18   <conf:(0.9)> lift:(2.14) lev:(0) conv:(3.86) 
```

### 題目 5

在美國由於聖誕節，12月是購物的旺季。請探勘分析比較12月與1~11月的顧客購物行為。有哪些相似的地方，有哪些差異的地方？ 

### 資料前處理